<img align="right" src="images/tf.png" width="128"/>
<img align="right" src="images/logo.png" width="128"/>
<img align="right" src="images/etcbc.png" width="128"/>
<img align="right" src="images/dans.png" width="128"/>

---

To get started: consult [start](start.ipynb)

---

# Similar lines

We spot the many similarities between lines in the corpus.

There are ca 50,000 lines in the corpus of which ca 35,000 with real content.
To compare these requires more than half a billion comparisons.
That is a costly operation.
[On this laptop it took 21 whole minutes](https://nbviewer.jupyter.org/github/etcbc/dss/blob/master/programs/parallels.ipynb).

The good news it that we have stored the outcome in an extra feature.

This feature is packaged in a TF data module,
that we will automatically loaded with the DSS.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import collections

from tf.app import use

In [3]:
A = use('dss', hoist=globals())

	connecting to online GitHub repo annotation/app-dss ... connected
Using TF-app in /Users/dirk/text-fabric-data/annotation/app-dss/code:
	rv0.4=#6f0198838fa9339fc85e23551bd83d61e4bbaaa7 (latest release)
	connecting to online GitHub repo etcbc/dss ... connected
Using data in /Users/dirk/text-fabric-data/etcbc/dss/tf/0.4:
	rv0.4=#dbdfeb914b510aa5bed5425b5b3096521e8b35c2 (latest release)
	connecting to online GitHub repo etcbc/dss ... connected
Using data in /Users/dirk/text-fabric-data/etcbc/dss/parallels/tf/0.4:
	rv0.4=#dbdfeb914b510aa5bed5425b5b3096521e8b35c2 (latest release)


The new feature is **sim** and it it an edge feature.
It annotates pairs of lines $(l, m)$ where $l$ and $m$ have similar content.
The degree of similarity is a percentage (between 60 and 100), and this value
is annotated onto the edges.

Here is an example:

In [8]:
exampleLine = F.otype.s('line')[0]
sisters = E.sim.b(exampleLine)
print(f'{len(sisters)} similar lines')
print('\n'.join(f'{s[0]} with similarity {s[1]}' for s in sisters[0:10]))
A.table(tuple((s[0],) for s in ((exampleLine,), *sisters)), end=10)

1 similar lines
1563765 with similarity 69


n,p,line
1,CD 1:1,ועתה שׁמעו כל יודעי צדק ובינו במעשׁי
2,4Q268 f1:9,ועתה שׁמעו ל׳י כול יודעי צדק ובינו במעשׁי אל ׃ כי ריב


# All similarities

Let's first find out the range of similarities:

In [9]:
minSim = None
maxSim = None

for l in F.otype.s('line'):
  sisters = E.sim.f(l)
  if not sisters:
    continue
  thisMin = min(s[1] for s in sisters)
  thisMax = max(s[1] for s in sisters)
  if minSim is None or thisMin < minSim:
    minSim = thisMin
  if maxSim is None or thisMax > maxSim:
    maxSim = thisMax

print(f'minimum similarity is {minSim:>3}')
print(f'maximum similarity is {maxSim:>3}')

minimum similarity is  60
maximum similarity is 100


# The bottom lines

We give a few examples of the least similar lines.

We can use a search template to get the 90% lines.

In [10]:
query = '''
line
-sim=60> line
'''

In words: find a line connected via a sim-edge with value 60 to an other line.

In [12]:
results = A.search(query)

  0.20s 24547 results


In [13]:
A.table(results, start=1, end=10)

n,p,line,line
1,4Q269 f2:4,בעדת׳ם ׃ ובני׳הם ב׳ו אבדו ומלכי׳הם ב׳ו נכרתו וגיבורי׳הם ב׳ו,אבדו ומלכי׳הם ב׳ו נכרתו וגבורי׳הם ב׳ו אבדו וארצ׳ם ב׳ו שׁממה ׃
2,4Q59 f2_3:1,אשׁר אמר יבוא עלי׳ך ועל עמ׳ך ועל בית אבי׳ך ימים אשׁר לא,יביא יהוה עלי׳ך ועל עמ׳ך ועל בית אבי׳ך ימים אשׁר לא באו למיום סור אפרים
3,4Q266 f8iii:6,ועשׁרים שׁנה עד בני שׁשׁים שׁנה ׃ ואל יתיצב עוד מבן,ובישׁודי הברית מבני חמשׁ ועשׁרים שׁנה ועד בן שׁשׁים שׁנה ׃ ואל יתיצב
4,4Q270 f6v:2,רחוק מן השׁער מלוא׳ו ׃ כי הוא אשׁר אמר שׁמור את,מן העת אשׁר יהיה גלגל השׁמשׁ רחוק מן השׁער מלוא׳ו כי הוא אשׁר אמר
5,4Q271 f5i:3,אם אלפים באמה ׃ אל ירם את יד׳ו להכות׳ה באגרוף אם,אל ירם אישׁ את יד׳ו להכות׳ה באגרוף ׃ אם סוררת היא אל יוציא׳ה
6,4Q270 f6v:19,וכל נפשׁ אדם אשׁר תפול אל מים מקום מים ואל מקום,הון ובצע בשׁבת ׃ וכל נפשׁ אדם אשׁר תפול אל מקום מים ואל בור אל
7,4Q266 f9ii:3,והעפר אשׁר יגואלו בטמאת האדם לגאולי שׁמן ב׳הם כפי,יגואלו בטמאת האדם לגאולי שׁמן ב׳הם כפי טמאת׳ם יטמא
8,4Q266 f9ii:4,טמאת׳ם יטמא הנוגע ב׳ם ׃ וכל כלי מסמר מסמר או יתד בכותל,הנוגע ב׳ם ׃ וכול כלי מסמר ויתד בכותל אשׁר יהיו עם
9,4Q267 f9iv:10,מבני המחנה להביא אישׁ אל העדה זולת פי המבקר אשׁר למחנה ׃,ימשׁול אישׁ מכול בני המחנה להביא אישׁ אל העדה
10,4Q269 f10ii:11,שׁלושׁת׳ם והגר רביע ׃ וכן ישׁבו וכן ישׁאלו לכל ׃ והכהן אשׁר יפקד,ישׁראל שׁלשׁיים והגר רביע ׃ וכן ישׁבו וכן ישׁאלו לכול ׃


Or in full layout:

In [14]:
A.table(results, start=1, end=10, fmt='layout-orig-full')

n,p,line,line
1,4Q269 f2:4,בעדת׳ם ׃ ובני׳הם ב׳ו אבדו ומלכי׳הם ב׳ו נכרתו וגיבורי׳הם ב׳ו,אבדו ומלכי׳הם ב׳ו נכרתו וגבורי׳הם ב׳ו אבדו וארצ׳ם ב׳ו שׁממה ׃
2,4Q59 f2_3:1,אשׁר אמר יבוא עלי׳ך ועל עמ׳ך ועל בית אבי׳ך ימים אשׁר לא,יביא יהוה עלי׳ך ועל עמ׳ך ועל בית אבי׳ך ימים אשׁר לא באו למיום סור אפרים
3,4Q266 f8iii:6,ועשׁרים שׁנה עד בני שׁשׁים שׁנה ׃ ואל יתיצב עוד מבן,ובישׁודי הברית מבני חמשׁ ועשׁרים שׁנה ועד בן שׁשׁים שׁנה ׃ ואל יתיצב
4,4Q270 f6v:2,רחוק מן השׁער מלוא׳ו ׃ כי הוא אשׁר אמר שׁמור את,מן העת אשׁר יהיה גלגל השׁמשׁ רחוק מן השׁער מלוא׳ו כי הוא אשׁר אמר
5,4Q271 f5i:3,אם אלפים באמה ׃ אל ירם את יד׳ו להכות׳ה באגרוף אם,אל ירם אישׁ את יד׳ו להכות׳ה באגרוף ׃ אם סוררת היא אל יוציא׳ה
6,4Q270 f6v:19,וכל נפשׁ אדם אשׁר תפול אל מים מקום מים ואל מקום,הון ובצע בשׁבת ׃ וכל נפשׁ אדם אשׁר תפול אל מקום מים ואל בור אל
7,4Q266 f9ii:3,והעפר אשׁר יגואלו בטמאת האדם לגאולי שׁמן ב׳הם כפי,יגואלו בטמאת האדם לגאולי שׁמן ב׳הם כפי טמאת׳ם יטמא
8,4Q266 f9ii:4,טמאת׳ם יטמא הנוגע ב׳ם ׃ וכל כלי מסמר מסמר או יתד בכותל,הנוגע ב׳ם ׃ וכול כלי מסמר ויתד בכותל אשׁר יהיו עם
9,4Q267 f9iv:10,מבני המחנה להביא אישׁ אל העדה זולת פי המבקר אשׁר למחנה ׃,ימשׁול אישׁ מכול בני המחנה להביא אישׁ אל העדה
10,4Q269 f10ii:11,שׁלושׁת׳ם והגר רביע ׃ וכן ישׁבו וכן ישׁאלו לכל ׃ והכהן אשׁר יפקד,ישׁראל שׁלשׁיים והגר רביע ׃ וכן ישׁבו וכן ישׁאלו לכול ׃


From now on we forget about the level of similarity, and focus on whether two lines are just "similar", meaning that they have
a high degree of similarity.

# Cluster the lines

Before we try to find them, let's see if we can cluster the lines in similar clusters.

In [15]:
CLUSTER_THRESHOLD = 0.5

def makeClusters():
  indent(reset=True)
  chunkSize = 1000
  b = 0
  j = 0
  clusters = []
  for l in F.otype.s('line'):
    j += 1
    b += 1
    if b == chunkSize:
      b = 0
      info(f'{j:>5} lines and {len(clusters):>5} clusters')
    lSisters = {x[0] for x in E.sim.b(l)}
    lAdded = False
    for cl in clusters:
      if len(cl & lSisters) > CLUSTER_THRESHOLD * len(cl):
        cl.add(l)
        lAdded = True
        break
    if not lAdded:
      clusters.append({l})
  info(f'{j} lines and {len(clusters)} clusters')
  return clusters

In [16]:
clusters = makeClusters()

  0.13s  1000 lines and   979 clusters
  0.47s  2000 lines and  1956 clusters
  1.03s  3000 lines and  2927 clusters
  1.80s  4000 lines and  3867 clusters
  2.81s  5000 lines and  4822 clusters
  3.99s  6000 lines and  5770 clusters
  5.37s  7000 lines and  6696 clusters
  6.94s  8000 lines and  7519 clusters
  8.64s  9000 lines and  8411 clusters
    10s 10000 lines and  9223 clusters
    12s 11000 lines and  9988 clusters
    14s 12000 lines and 10761 clusters
    17s 13000 lines and 11522 clusters
    19s 14000 lines and 12330 clusters
    22s 15000 lines and 13270 clusters
    25s 16000 lines and 14110 clusters
    28s 17000 lines and 14997 clusters
    32s 18000 lines and 15776 clusters
    35s 19000 lines and 16553 clusters
    39s 20000 lines and 17369 clusters
    42s 21000 lines and 18178 clusters
    46s 22000 lines and 18926 clusters
    50s 23000 lines and 19778 clusters
    54s 24000 lines and 20657 clusters
    59s 25000 lines and 21487 clusters
 1m 04s 26000 lines and 2

What is the distribution of the clusters, in terms of how many similar lines they contain?
We count them.

In [17]:
clusterSizes = collections.Counter()

for cl in clusters:
  clusterSizes[len(cl)] += 1
  
for (size, amount) in sorted(
  clusterSizes.items(),
  key=lambda x: (-x[0], x[1]),
):
  print(f'clusters of size {size:>4}: {amount:>5}')

clusters of size   99:     1
clusters of size   70:     1
clusters of size   54:     1
clusters of size   52:     1
clusters of size   45:     1
clusters of size   42:     2
clusters of size   36:     1
clusters of size   33:     2
clusters of size   32:     1
clusters of size   27:     3
clusters of size   25:     2
clusters of size   24:     1
clusters of size   21:     2
clusters of size   20:     1
clusters of size   18:     1
clusters of size   17:     3
clusters of size   16:     3
clusters of size   15:     2
clusters of size   14:     3
clusters of size   13:     8
clusters of size   12:    11
clusters of size   11:    12
clusters of size   10:    17
clusters of size    9:    15
clusters of size    8:    24
clusters of size    7:    31
clusters of size    6:    59
clusters of size    5:    84
clusters of size    4:   246
clusters of size    3:   600
clusters of size    2:  4128
clusters of size    1: 39055


# Interesting groups

Exercise: investigate some interesting groups, that lie in some sweet spots.

* the biggest clusters: more than 13 members
* the medium clusters: between 5 and 13 members
* the small clusters: between 2 and 4 members

---

All chapters:

* **[start](start.ipynb)** become an expert in creating pretty displays of your text structures
* **[display](display.ipynb)** become an expert in creating pretty displays of your text structures
* **[search](search.ipynb)** turbo charge your hand-coding with search templates
* **[exportExcel](exportExcel.ipynb)** make tailor-made spreadsheets out of your results
* **[share](share.ipynb)** draw in other people's data and let them use yours
* **similarLines** spot the similarities between lines

---

See the [cookbook](cookbook) for recipes for small, concrete tasks.